# **Importing Models!**

In [0]:
!git clone https://github.com/NightFury010497/Udacity_Data_Scientist_NanoDegree
!cd /content/Udacity_Data_Scientist_NanoDegree/First_Blog/

In [0]:
!unzip /content/models.zip

In [0]:
import os
import cv2
import time
import torch
import argparse
import numpy as np
import torch.backends.cudnn as cudnn
from PIL import Image
from models.retinaface.config import cfg
from models.retinaface.prior_box import PriorBox
from models.retinaface.py_cpu_nms import py_cpu_nms
from models.retinaface.retinaface import RetinaFace
from models.retinaface.box_utils import decode , decode_landm


In [0]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Setting up the Paths!

In [0]:
trained_model_path = "/content/models/retinaface/weights/Final_Retinaface.pth"
cpu = True
confidence_threshold = 0.05
top_k = 5000
nms_threshold = 0.3
keep_top_k = 750
save_image_path = "/content/predictedimg"
vis_threshold = 0.6

In [0]:
print(device)

# Loading Model!

In [0]:
### check_keys

def check_keys(model, pretrained_state_dict):
    '''In this function we will be checking keys of the pretrained model!'''
    ckpt_keys = set(pretrained_state_dict.keys())
    model_keys = set(model.state_dict().keys())
    used_pretrained_keys = model_keys & ckpt_keys
    unused_pretrained_keys = ckpt_keys - model_keys
    missing_keys = model_keys - ckpt_keys
    print('Missing keys:{}'.format(len(missing_keys)))
    print('Unused checkpoint keys:{}'.format(len(unused_pretrained_keys)))
    print('Used keys:{}'.format(len(used_pretrained_keys)))
    assert len(used_pretrained_keys) > 0, 'load NONE from pretrained checkpoint'
    return True

### remove_prefix
def remove_prefix(state_dict, prefix):
    ''' Old style model is stored with all names of parameters sharing common prefix 'module.' '''
    print('remove prefix \'{}\''.format(prefix))
    f = lambda x: x.split(prefix, 1)[-1] if x.startswith(prefix) else x
    return {f(key): value for key, value in state_dict.items()}


### load_model
def load_model(model, pretrained_path, load_to_cpu):
    print('Loading pretrained model from {}'.format(pretrained_path))
    if load_to_cpu:
        pretrained_dict = torch.load(pretrained_path, map_location=lambda storage, loc: storage)
    else:
        device = torch.cuda.current_device()
        pretrained_dict = torch.load(pretrained_path, map_location=lambda storage, loc: storage.cuda(device))
    if "state_dict" in pretrained_dict.keys():
        pretrained_dict = remove_prefix(pretrained_dict['state_dict'], 'module.')
    else:
        pretrained_dict = remove_prefix(pretrained_dict, 'module.')
    check_keys(model, pretrained_dict)
    model.load_state_dict(pretrained_dict, strict=False)
    return model

In [0]:
torch.set_grad_enabled(False)

#net and model
net = RetinaFace(phase="test")
net = load_model(net , trained_model_path, cpu)
net.eval()
print("Finished loading model!")
cudnn.benchmark = True
device = torch.device("cpu" if cpu else "cuda")
net = net.to(device)

In [0]:
resize = 1
image_path = "/content/20065352_263001610848524_1703808604290678784_n.jpg"
save_image = True

In [0]:
name = image_path.split('/')[-1].split('.')[-2]
name

# Making Function to Recognize Faces through bounding boxes!


In [0]:
def recog(path):
    ''' Creating a Function to Recognize!'''
    image_path = path
    
    img_raw = cv2.imread(image_path, cv2.IMREAD_COLOR)

    img = np.float32(img_raw)

    im_height, im_width, _ = img.shape
    scale = torch.Tensor([img.shape[1], img.shape[0], img.shape[1], img.shape[0]])
    img -= (104, 117, 123)
    img = img.transpose(2, 0, 1)
    img = torch.from_numpy(img).unsqueeze(0)
    img = img.to(device)
    scale = scale.to(device)

    tic = time.time()
    loc, conf, landms = net(img)  # forward pass
    print('net forward time: {:.4f}'.format(time.time() - tic))

    priorbox = PriorBox(cfg, image_size=(im_height, im_width))
    priors = priorbox.forward()
    priors = priors.to(device)
    prior_data = priors.data
    boxes = decode(loc.data.squeeze(0), prior_data, cfg['variance'])
    boxes = boxes * scale / resize
    boxes = boxes.cpu().numpy()
    scores = conf.squeeze(0).data.cpu().numpy()[:, 1]
    landms = decode_landm(landms.data.squeeze(0), prior_data, cfg['variance'])
    scale1 = torch.Tensor([img.shape[3], img.shape[2], img.shape[3], img.shape[2],
                                   img.shape[3], img.shape[2], img.shape[3], img.shape[2],
                                   img.shape[3], img.shape[2]])
    scale1 = scale1.to(device)
    landms = landms * scale1 / resize
    landms = landms.cpu().numpy()

    # ignore low scores
    inds = np.where(scores > confidence_threshold)[0]
    boxes = boxes[inds]
    landms = landms[inds]
    scores = scores[inds]

    # keep top-K before NMS
    order = scores.argsort()[::-1][:top_k]
    boxes = boxes[order]
    landms = landms[order]
    scores = scores[order]

    # do NMS
    dets = np.hstack((boxes, scores[:, np.newaxis])).astype(np.float32, copy=False)
    keep = py_cpu_nms(dets, nms_threshold)
    # keep = nms(dets, args.nms_threshold,force_cpu=args.cpu)
    dets = dets[keep, :]
    landms = landms[keep]

    # keep top-K faster NMS
    dets = dets[:keep_top_k, :]
    landms = landms[:keep_top_k, :]

    dets = np.concatenate((dets, landms), axis=1)

    # show image
    if save_image:
                for b in dets:
                    if b[4] < vis_threshold:
                        continue
                    text = "{:.4f}".format(b[4])
                    b = list(map(int, b))
                    cv2.rectangle(img_raw, (b[0], b[1]), (b[2], b[3]), (0, 0, 255), 2)
                    cx = b[0]
                    cy = b[1] + 12
                    cv2.putText(img_raw, text, (cx, cy),
                                cv2.FONT_HERSHEY_DUPLEX, 0.5, (255, 255, 255))
                    # df3.append(boxes)
                    # df[0] = df[0].append(b[0])
                    df2 = b[:4]

                    # print(boxes)
                    # print('+++++++++++++++++++++')
                    # print(b)

                    # landms
                    #cv2.circle(img_raw, (b[5], b[6]), 4, (0, 0, 255), 4)
                    #cv2.circle(img_raw, (b[7], b[8]), 4, (0, 255, 255), 4)
                    #cv2.circle(img_raw, (b[9], b[10]), 4, (255, 0, 255), 4)
                    #cv2.circle(img_raw, (b[11], b[12]), 4, (0, 255, 0), 4)
                    #cv2.circle(img_raw, (b[13], b[14]), 4, (255, 0, 0), 4)
                # save image
                name = image_path.split('/')[-1].split('.')[-2]
                cv2.imwrite("{}/{}.png".format(save_image_path, name), img_raw)
    im = Image.open("{}/{}.png".format(save_image_path, name))
    return im


# Testing Time!

In [0]:
recog("/content/20065352_263001610848524_1703808604290678784_n.jpg")

In [0]:
recog("/content/19985624_375648532837845_2728904469055012864_n.jpg")

In [0]:
recog("/content/23594899_304976046684057_3095936779144921088_n.jpg")